<a href="https://colab.research.google.com/github/Surabhi6300/Speech_Text_HuggingFace/blob/main/Hugging_face_Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install transformers

In [88]:
!pip install transformers torchaudio torch

In [89]:
from transformers import pipeline

In [90]:
import transformers
print(transformers.__version__)

4.56.1


In [91]:
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np

In [92]:
processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
from google.colab import output
from base64 import b64decode
import IPython.display as display

def record_audio(filename="my_audio.wav"):
    js = """
    async function record(sec) {
        const stream = await navigator.mediaDevices.getUserMedia({audio: true});
        const recorder = new MediaRecorder(stream);
        let data = [];
        recorder.ondataavailable = event => data.push(event.data);
        recorder.start();
        await new Promise(r => setTimeout(r, sec * 1000));
        recorder.stop();
        await new Promise(r => recorder.onstop = r);
        let audio = new Blob(data);
        let reader = new FileReader();
        reader.readAsDataURL(audio);
        reader.onloadend = () => {
            google.colab.kernel.invokeFunction('notebook.recorded', [reader.result], {});
        };
    }
    record(5)  // Record for 5 seconds
    """
    output.register_callback('notebook.recorded', lambda s: open(filename, "wb").write(b64decode(s.split(',')[1])))
    display.display(display.Javascript(js))

record_audio("my_audio.wav")


<IPython.core.display.Javascript object>

In [94]:
import librosa

# Load the file that was saved
audio, sampling_rate = librosa.load("my_audio.wav", sr=16000)

print("Shape:", audio.shape)
print("Sample Rate:", sampling_rate)


Shape: (79680,)
Sample Rate: 16000


/tmp/ipython-input-4009978104.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sampling_rate = librosa.load("my_audio.wav", sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [95]:
display.Audio('my_audio.wav', autoplay=True)

In [96]:
input_values = processor(audio,sampling_rate=16000,return_tensors = "pt").input_values

input_values

tensor([[-0.0003, -0.0003, -0.0003,  ..., -0.0021, -0.0045, -0.0046]])

In [97]:
logits= model(input_values).logits
logits

tensor([[[ 12.8271, -26.1798, -25.9160,  ...,  -6.6918,  -7.3308,  -7.2776],
         [ 12.9298, -26.3921, -26.1114,  ...,  -6.7743,  -7.4865,  -7.3200],
         [ 12.6997, -26.0345, -25.7571,  ...,  -6.7451,  -7.2747,  -7.4073],
         ...,
         [ 12.9344, -25.9628, -25.6859,  ...,  -6.8147,  -7.7617,  -7.4507],
         [ 12.7867, -25.9830, -25.7229,  ...,  -6.8111,  -7.7621,  -7.5583],
         [ 12.7469, -26.2092, -25.9582,  ...,  -6.9627,  -8.0023,  -7.4365]]],
       grad_fn=<ViewBackward0>)

In [98]:
predicted_ids = torch.argmax(logits, dim = -1)

In [99]:
transcriptions = processor.decode(predicted_ids[0])
transcriptions

'AND WILL THIS AS SOONER BE HOW ARE YOU DOING TO DAY'